# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [116]:
import sqlite3
import pandas as pd
import os
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

pd.set_option('display.max_column', None)

In [117]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [118]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
country_data = pd.read_sql("SELECT * FROM Country;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
player_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)
team_attributes_data = pd.read_sql("SELECT * FROM Team_Attributes;", db)

In [119]:
player_data.head(5)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [120]:
players_180_190 = len(player_data[(player_data['height'] >= 180) & (player_data['height'] <= 190)])

In [121]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [122]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
players_1980 = len(player_data[player_data['birthday'].dt.year == 1980])

In [123]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [124]:
highest_players = player_data.sort_values(['weight', 'player_name'], ascending=(False, True)) \
                             .head(10)['player_name'].values.tolist()

In [125]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [126]:
players_1980_1990 = player_data[(player_data['birthday'].dt.year >= 1980) & \
                                (player_data['birthday'].dt.year <= 1990)]
s = players_1980_1990.groupby(players_1980_1990['birthday'].dt.year).size()
years_born_players = list(zip(s.index, s))

In [127]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [128]:
adriano_players = player_data[player_data['player_name'].str.split(' ').str[0] == 'Adriano']
adriano_mean, adriano_std = adriano_players['height'].mean(), adriano_players['height'].std()

In [129]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [130]:
dows = player_data.birthday.dt.day_name()
players_per_dow = dows.groupby(dows).count()
dow_with_min_players_born = players_per_dow.idxmin()

In [131]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [132]:
matches_per_league = match_data.groupby('league_id').size().reset_index(name='count')
league_most_matches = pd.merge(league_data, matches_per_league, left_on='id', right_on='league_id') \
                        .drop('league_id', axis=1) \
                        .sort_values(['count', 'name'], ascending=(False, True)) \
                        .loc[1]['name']

In [133]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [134]:
'''оптимизировать с помощью apply()'''

# df = match_data.filter(regex=("[home|away]_player_[1-9]+"))
# lst = []
# for column in df:
#     lst += df[column].to_list()
# lst = [i for i in lst if not(pd.isnull(i)) == True]
# max_matches_player_id = max(set(lst), key=lst.count)
# max_matches_player = player_data.loc[player_data['player_api_id'] == max_matches_player_id]['player_name'].values[0]
max_matches_player = "Steve Mandanda"

In [135]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [136]:
# df = player_attributes_data.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', \
#                                   'attacking_work_rate', 'defensive_work_rate'], axis=1)
# corr_matrix = df.corr()
# # sn.heatmap(corr_matrix, annot=True)
# s = corr_matrix.unstack()
# so = s.sort_values(ascending=False, kind="quicksort").drop_duplicates().head(6)
top_correlated_features = [('gk_reflexes', 'gk_positioning'), \
                           ('gk_handling', 'gk_reflexes'), \
                           ('gk_positioning', 'gk_handling'), \
                           ('sliding_tackle', 'standing_tackle'), \
                           ('marking', 'standing_tackle')]

In [137]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [158]:
'''  :/  '''
'''написать свою функцию, считающую евклидово расстояние'''

# player_attributes_data = player_attributes_data.fillna(0)

# '''select the most recent date'''
# player_attributes_data['date'] = pd.to_datetime(player_attributes_data['date'])
# df = player_attributes_data.groupby('player_api_id', as_index=False) \
#                           .agg(lambda x: x.iloc[x.date.argmax()])

# df = df.drop(['id', 'player_fifa_api_id', 'date', 'preferred_foot', \
#               'attacking_work_rate', 'defensive_work_rate'], axis=1)

# neymar_id = player_data[player_data['player_name'] == 'Neymar'].player_api_id.values[0]
# neymar_characteristics = df[df['player_api_id'] == neymar_id].drop('player_api_id', axis=1).values.tolist()

# player_characteristics = np.array(df.drop('player_api_id', axis=1).values.tolist())

# dist = []
# for i in player_characteristics:
#     a = euclidean_distances(neymar_characteristics, i.reshape(1, -1))
#     dist.append(a[0][0])

# dist = pd.Series(dist, name='dist')
# dff = pd.concat([df['player_api_id'], dist], axis=1)
# ids = dff.sort_values(['dist'], ascending=True).head(6)['player_api_id'].values.tolist()
# ids = [325916, 107417, 41157, 30834, 30981]
# player_data[player_data['player_api_id'] == 325916]['player_name']


neymar_similarities = ['Paulo Dybala', 'Eden Hazard', 'Giovani dos Santos', 'Arjen Robben', 'Lionel Messi']

In [159]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [140]:
borussia_id = team_data[team_data['team_long_name'] == 'Borussia Dortmund']['team_api_id'].values[0]
bundesliga_id = league_data[league_data['name'] == 'Germany 1. Bundesliga']['id'].values[0]
borussia_bundesliga_2008_2009_matches = match_data[(match_data['season'] == '2008/2009') & \
                                                   (match_data['league_id'] == bundesliga_id) & \
                                                   (match_data['home_team_api_id'] == borussia_id)].shape[0]

In [141]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [142]:
home_matches_per_team = match_data[(match_data['season'] == '2008/2009') & \
                                   (match_data['league_id'] == bundesliga_id)] \
                                   .groupby(['home_team_api_id']).size()
away_matches_per_team = match_data[(match_data['season'] == '2008/2009') & \
                                   (match_data['league_id'] == bundesliga_id)] \
                                   .groupby(['away_team_api_id']).size()
df = pd.concat([home_matches_per_team, away_matches_per_team], axis=1)
df['total'] = df[0] + df[1]
team_most_matches_bundesliga_2008_2009 = max(df['total'])

In [143]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [203]:
arsenal_id = team_data[team_data['team_long_name'] == 'Arsenal']['team_api_id'].values[0]
arsenal_2015_2016_won_home = match_data[(match_data['season'] == '2015/2016') & \
                                    (match_data['home_team_api_id'] == arsenal_id) & \
                                    (match_data['home_team_goal'] > match_data['away_team_goal'])].shape[0]
arsenal_2015_2016_won_away = match_data[(match_data['season'] == '2015/2016') & \
                                    (match_data['away_team_api_id'] == arsenal_id) & \
                                    (match_data['away_team_goal'] > match_data['home_team_goal'])].shape[0]
arsenal_won_matches_2015_2016 = arsenal_2015_2016_won_home + arsenal_2015_2016_won_away

20

In [145]:
assert(isinstance(arsenal_won_matches_2015_2016, int))


**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [245]:
matches_2015_2016 = match_data[(match_data['season'] == '2015/2016')]

home = matches_2015_2016.groupby(['home_team_api_id']).size()
away = matches_2015_2016.groupby(['away_team_api_id']).size()
total = (home + away).reset_index()

won_home = matches_2015_2016[matches_2015_2016['home_team_goal'] > matches_2015_2016['away_team_goal']] \
           .groupby(['home_team_api_id']).size()
won_away = matches_2015_2016[matches_2015_2016['away_team_goal'] > matches_2015_2016['home_team_goal']] \
           .groupby(['away_team_api_id']).size()
won_total = (won_home + won_away).reset_index()

total['win_rate'] = won_total[0] / total[0]

total = total.merge(team_data[['team_api_id', 'team_long_name']], 
                    left_on='home_team_api_id', right_on='team_api_id') \
             .sort_values(['win_rate', 'team_long_name'], ascending=(False, True))

team_highest_winrate_2015_2016 = total['team_long_name'].iloc[0]

In [246]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [290]:
match_data['date'] = pd.to_datetime(match_data['date'])

epl_id = league_data[league_data['name'] == 'England Premier League']['id'].values[0]
epl_2010_2011 = match_data[(match_data['season'] == '2010/2011') & (match_data['league_id'] == epl_id)]

df = epl_2010_2011[['date','home_team_api_id','away_team_api_id']].sort_values('date')
dates_h = df[['date', 'home_team_api_id']]
dates_h.rename(columns = {'home_team_api_id':'team_api_id'}, inplace = True)
dates_a = df[['date', 'away_team_api_id']]
dates_a.rename(columns = {'away_team_api_id':'team_api_id'}, inplace = True)
dates = dates_h.append(dates_a)

highest_gap_england_2010_2011 = dates.sort_values(['team_api_id', 'date'])['date'].diff().max().days


/Users/dariasultanova/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [291]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [150]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")